# Inferential statistics
## Part III - Inferential Analysis

We're now going to look for answers to the ongoing basketball discussions between you and your family. The main ones we want to reasearch are the following:

- Your grandmother says that your sister couldn't play in a professional basketball league (not only the WNBA, but ANY professional basketball league) because she's too skinny and lacks muscle.
- Your sister says that most female professional players fail their free throws.
- Your brother-in-law heard on the TV that the average assists among NBA (male) and WNBA (female) players is 52 for the 2016-2017 season. He is convinced this average would be higher if we only considered the players from the WNBA.

Let's investigate these claims and see if we can find proof to refute or support them.

### Libraries
Import the necessary libraries first.

In [1]:
# Libraries
import math
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
pd.set_option('max_columns', 50)

### Load the dataset

Load the cleaned dataset.

In [2]:
#your code here
wnba = pd.read_csv("../data/wnba_clean.csv", index_col = 0)

# Question 1: Can my sister play in a professional female basketball league?

As we said, you grandmother is convinced that your sister couldn't play in a professional league because of her physique and weight (her weight is 67kg). 

To find an actual answer to the question we first need to know what's the average weight of a professional female basketball player. The data we have only refers to the WNBA league and not to every female professional basketball league in the world, therefore we have no way of actually calculating it.

Still, given that we do have *some* data we can **infer** it using a sample of players like the one we have. 

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the average weight. Do you feel it actually fulfills those requirements? Do you need to make any assumptions? We could calculate a confidence interval to do the inference, but do you know any other ways?**

In [4]:
# your answer here
"""
How would you do it?
    I do not know enough to know. 

Do you feel it actually fulfills those requirements?
    I do not know enough to know. 

Do you need to make any assumptions?
    As far as I know, there's no situation where we're not making assumptions.

We could calculate a confidence interval to do the inference, but do you know any other ways?
    Probably there are other ways..? So I don't know, but I think that there are..?
"""

"\nHow would you do it?\n    I do not know enough to know. \n\nDo you feel it actually fulfills those requirements?\n    I do not know enough to know. \n\nDo you need to make any assumptions?\n    As far as I know, there's no situation where we're not making assumptions.\n\nWe could calculate a confidence interval to do the inference, but do you know any other ways?\n    Probably there are other ways..? So I don't know, but I think that there are..?\n"

**Now that all the requirements have been taken into account, compute the confidence interval of the average weight with a confidence level of 95%.**

In [6]:
# your code here
# this?
ci1 = stats.t.interval(alpha = 0.95, df = len(wnba) - 1, loc = wnba["Weight"].mean(), 
                      scale = wnba["Weight"].std() / np.sqrt(len(wnba) - 1))

# or this?
ci2 = stats.norm.interval(0.95, loc = wnba["Weight"].mean(), 
                          scale = stats.sem(wnba["Weight"], ddof = 1))

# They are very close (at least apparently), but they are not the same. :/
print(ci1, ci2)

(77.14815649242229, 80.80958998645096) (77.17027122332428, 80.78747525554897)


**What can you say about these results?**

In [10]:
#your-answer-here
"""
That, with 95% confidence (whatever that means), the average weight of the WNBA players is 
between 77.15Kg and 80.81Kg (or between 77.17Kg and 80.79Kg)?
"""

**If your sister weighs 67kg what would you tell your grandmother in regards to her assumption?**

In [31]:
#your-answer-here
"""
I'd say (without much confidance) that, while her weight seems to be below average for a pro 
female player, she wouldn't be the one with the lowest weight, that 19% of the players have
less than 68Kgs, which is about 19% of the players in the WNBA, to say nothing of having NOT
been demonstrated any relation between weight and skill/ability.
"""

# Actually these, I feel, give a better answer than the average, but that might be wrong on 
# my part.
print(f"Lowest weight in WNBA: {wnba['Weight'].min()}Kgs")
print("Amount of players with less than 68Kgs:", len(wnba.loc[wnba["Weight"] < 68]))
print(f"Percentage of players with less than 68Kgs: {round((len(wnba.loc[wnba['Weight'] < 68]) / len(wnba))* 100, 2)}%")

Lowest weight in WNBA: 55Kgs
Amount of players with less than 68Kgs: 27
Percentage of players with less than 68Kgs: 19.01%


## Bonus: Can you plot the probability distribution of the average weight, indicating where the critical region is?

In [8]:
# your code here
"""
No, I cannot; maybe I can draw-ish (not to scale)?
"""

# Question 2: Do female professional basketball players fail the majority of their free throws?

You do not agree with your sister when she says that most female players fail their free throws. You decide to try and estimate the percentage of players that fail more than 40% of their free throws using, you guessed it, the WNBA sample.

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the proportion of players that miss more than 40% of their free throws. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [63]:
# your answer here
"""
«You decide to try and estimate the percentage of players that fail more than 40% of their 
free throws» 

I wouldn't (do it this way). 
I don't know about the other questions (except that, as far as I know, I think we always need 
to make assumptions). 

That said, I'd just check the relationship between the Free Throws Made and the Free Throws 
Attempeted (see below). Then I have a problem: I'm not actually answering the question. I mean,
I'm answering some other question(s), but not the question of the majority of the players 
failling (or not) their free throws... 

...So... What I could do was to create a new column that, for each player, it calculated the 
percentage of FTM's by FTA's, then count the number of players that had a percentage higher 
than 50%, divide than by the total number of players, and check if that number is higher than
50% of the population (or not). This could be done with groupby/pivot_table to speed up the 
process, but yes, I am very skeptical (to put it midly) that this would be the best way.

...So: I don't understand why chose 40% and not 50%, but taking it, what could I do? My sample
would have to be well spread (meaning, would have to have a frequency distribuition very very
similar to the whole population), at least in regards to the FTM's and FTA's (I don't know/ 
understand what FT% is measuring). I don't know what else. And I don't know what assumptions
I'd be making (or am making), only that I'm confident that I am making some (even without
knowing what they are), and I have no idea if I 'feel'that the sample satisfies that 
requirement or not (I don't know the sample yet). :/
"""

print("Average Free Throws Made:", round(wnba["FTM"].mean(), 2))
print("Average Free Throws Attempted", round(wnba["FTA"].mean(), 2))
print("I don't understand this Stat (Free throws percentage?? percentage of what?)", 
      round(wnba["FT%"].mean(), 2))
print("Another way to check the FTM mean", round(wnba["FTM"].sum() / len(wnba["FTM"]), 2))
print("Relationship between FTM and FTA:", round(wnba["FTM"].sum() / wnba["FTA"].sum(), 2))


Average Free Throws Made: 39.54
Average Free Throws Attempted 49.42
I don't understand this Stat (Free throws percentage?? percentage of what?) 75.83
Another way to check the FTM mean 39.54
Relationship between FTM and FTA: 0.8


**Now that all the requirements have been taken into account, compute the confidence interval of the proportion with a confidence level of 95%:**

In [12]:
# your code here
"""
I don't know what to do anymore.
"""

**What can you comment about our result? What would you tell your sister?**

In [15]:
#your-answer-here

# Bonus: Can you plot the probability distribution of the proportion of missed free throws, indicating where the critical region is?

In [ ]:
#your code here

# Question 3: Is the average number of assists for WNBA players only higher than the average for WNBA and NBA players together?

Your brother-in-law is convinced that the average assists for female professional players is higher than the average of both female and male players combined (which is 52 for the 2016-2017 season). You would like to actually prove if this is true or not but you remember your stats teacher saying "you can't *prove* anything, you just can say that *you are not* saying foolishness".

**How would you do it? Try and think about the requirements that your sample must satisfy in order to do that. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [16]:
#your-answer-here

**Use a two-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [15]:
#your code here

In [18]:
#your-answer-here

**Now use a one-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [20]:
#your-answer-here

# Bonus: Can you plot the resulting t-distribution of both tests? Indicate where the is the critical region and where does your statistic fall.**

In [ ]:
#your code here

# Bonus: Satisfying your curiosity

You finally managed to solve your family's debates over basketball! While you were doing that you started to take an interest in the normal distribution.

You read that the normal distribution is present in a lot of natural phenomenons, like blood pressure, IQ, weight and height. If, for example, we could plot the distribution of the weights of every human on the planet right now it would have the shape of a normal distribution.

In light of this you would like to see if it's possible to check if the distribution of the weights of the WNBA players is a sample distribution that comes from a population that has a normal distribution, because theoretically this should be the case.

**How would you try to demonstrate that our sample fits a normal distribution? What kind of test would you use? Would you have to make any assumptions?**

In [22]:
#your-answer-here

In [19]:
# your code here

**What are your comments in regards to the results of the test?**

In [24]:
#your-answer-here